## Clone Source Code

In [1]:
# !git clone https://github.com/asusilo17/VehicleDetection.git

### Copy Object Script

In [2]:
# !cp -r /content/VehicleDetection/*.py /content/
# !cp -r /content/VehicleDetection/Model/ /content/

## Copy From Google Drive

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

### Copy Data Video

In [4]:
# !cp -r /content/drive/MyDrive/UNPAM/ANPR_Video_Cut/ /content/DataVideo/

### Copy Yolo Model

In [5]:
# !cp -r /content/drive/MyDrive/UNPAM/VehicleDetection/Yolo/ /content/Yolo/

## Install library

In [6]:
# !pip install pyodbc opencv-python pandas pyodbc ultralytics easyocr roboflow numpy dotenv pytesseract
# !pip install paddlepaddle==2.5.2
# !pip install paddleocr==2.10.0

In [7]:
import cv2
import torch
from paddleocr import PaddleOCR
from ultralytics import YOLO
import numpy as np
from datetime import datetime
from dotenv import load_dotenv
import os
import glob
from Model import imageProcess as ip
from Model import ocrProcess as op
from Model import saveToFile as csv
from Model import sqlProcess as sqlp


d:\Mine\UNPAM\THESIS\VehicleDetection\.venv\Lib\site-packages\paddle\utils\cpp_extension\extension_utils.py:711: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)


[2025/07/15 21:26:26] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\susil/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\susil/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320',

[2025-07-15 21:26:27,905] [ WARNING] easyocr.py:80 - Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


conn_str : DRIVER={ODBC Driver 17 for SQL Server};SERVER=ASUS110\SQLEXPRESS;DATABASE=ANPR;UID=sa;PWD=sa


In [8]:

# Load environment variables
load_dotenv()
iot_location_id = os.getenv("iot_location_id")
bot_token = os.getenv('bot_token')
bot_uname = os.getenv('bot_uname')
bot_name = os.getenv('bot_name')

ocr = PaddleOCR(use_angle_cls=True, lang='en')


yolo_model = [
    {"version": "Small", "object":"yolov10s", "license_plat":"best_yolov10s_licenseplat"},
    {"version": "Nano", "object":"yolov10n", "license_plat":"best_yolov10n_licenseplat"},
    {"version": "Medium", "object":"yolov10m", "license_plat":"best_yolov10m_licenseplat"},
    {"version": "Balance", "object":"yolov10b", "license_plat":"best_yolov10b_licenseplat"},
    {"version": "Large", "object":"yolov10l", "license_plat":"best_yolov10l_licenseplat"},
    {"version": "Extra Large", "object":"yolov10x", "license_plat":"best_yolov10x_licenseplat"},
]

[2025/07/15 21:26:34] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\susil/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\susil/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320',

In [9]:
today = datetime.today().strftime('%Y-%m-%d')

# Load models
YoloModelVehicle = YOLO(f"Yolo/yolov10s.pt")
YoloModelLicenseNumber = YOLO(f"Yolo/best_yolov10s_licenseplat.pt")


# Config
skip_frames = 5  # Proses setiap 5 frame
rtsp_url = 'DataVideo/A1206YV.mp4'

cap = cv2.VideoCapture(rtsp_url)
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    if frame_count % skip_frames != 0:
        continue

    vehicle_mask = np.zeros(frame.shape[:2], dtype=np.uint8)
    original_frame = frame.copy()

    # Deteksi kendaraan
    vehicle_results = YoloModelVehicle(frame)
    for vehicle in vehicle_results:
        for box in vehicle.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
            class_id = int(box.cls[0])
            confidence = box.conf[0]

            if class_id in [2, 3, 5, 7] and confidence > 0.50:
                vehicle_crop = frame[y1:y2, x1:x2]
                vh, vw = vehicle_crop.shape[:2]
                vehicle_crop_resized = cv2.resize(vehicle_crop, (640, 640))

                # ip.f_image_read(frame, vehicle_crop, "Original", "Kendaraan")

                # Deteksi plat nomor
                license_plate_results = YoloModelLicenseNumber(vehicle_crop)
                for result in license_plate_results:
                    for lp_box in result.boxes:
                        lx1, ly1, lx2, ly2 = map(int, lp_box.xyxy[0].cpu().numpy())
                        lp_conf = lp_box.conf[0]

                        license_crop = vehicle_crop[ly1:ly2, lx1:lx2]

                        # ip.f_image_read(vehicle_crop, license_crop, "Kendaraan", "Plat")

                        # if lp_conf > 0.1:
                        #     scale_x = vw / 640
                        #     scale_y = vh / 640
                        #     lx1 = int(lx1 * scale_x)
                        #     lx2 = int(lx2 * scale_x)
                        #     ly1 = int(ly1 * scale_y)
                        #     ly2 = int(ly2 * scale_y)

                        #     abs_lx1 = lx1 + x1
                        #     abs_lx2 = lx2 + x1
                        #     abs_ly1 = ly1 + y1
                        #     abs_ly2 = ly2 + y1

                            # license_crop = original_frame[abs_ly1:abs_ly2, abs_lx1:abs_lx2]

                            # OCR
                        plat_number, plat_ori, plat_conf = op.f_ocr_process(license_crop)
                        if plat_number:
                            # Jika ingin anotasi box disimpan ke file atau log, bisa disimpan di sini
                            # frame = f_image_box_license_2(frame, abs_lx1, abs_lx2, abs_ly1, abs_ly2, plat_number)
                            nama_file = rtsp_url.replace("DataVideo/",'')
                            nama_file = nama_file.replace('.mp4','')

                            # plat_number = op.correct_ocr_plate(plat_number)

                            print(f"nama_file: {nama_file} | {plat_ori}")

                            # csv.f_savePlateIdentification_to_csv(
                            sqlp.f_sql_savePlateIdentification(
                                p_file_name=nama_file,
                                p_license_number=plat_number,
                                p_confidence=plat_conf,
                                p_label=plat_ori,
                                p_metode="small"
                            )

cap.release()



0: 384x640 1 car, 428.7ms
Speed: 12.6ms preprocess, 428.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 512x640 2 License_Plates, 473.0ms
Speed: 10.1ms preprocess, 473.0ms inference, 0.9ms postprocess per image at shape (1, 3, 512, 640)
[2025/07/15 21:26:41] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.27557826042175293
[2025/07/15 21:26:41] ppocr DEBUG: cls num  : 1, elapsed : 0.21028995513916016
[2025/07/15 21:26:42] ppocr DEBUG: rec_res num  : 1, elapsed : 0.41881632804870605
nama_file: A1206YV | 0206V
[2025/07/15 21:26:42] ppocr DEBUG: dt_boxes num : 0, elapsed : 0.28343749046325684
[2025/07/15 21:26:42] ppocr DEBUG: cls num  : 0, elapsed : 0
[2025/07/15 21:26:42] ppocr DEBUG: rec_res num  : 0, elapsed : 0.0
Gagal dapat hasil OCR

0: 384x640 1 car, 1 truck, 599.7ms
Speed: 9.3ms preprocess, 599.7ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 512x640 1 License_Plate, 767.2ms
Speed: 7.7ms preprocess, 767.2ms inference, 1.1ms postprocess